In [3]:
import numpy as np

# Уравнение теплопроводности

Начально-краевая задача для уравнения теплопроводности с постоянным коэффициентом в общем виде можно записать следующим образом:
$$
\begin{array}{l}
\dfrac{\partial u}{\partial t} = \alpha \dfrac{\partial^2 u}{\partial x^2} + f(x, t), \; t > 0, \; x \in (0, l_x), \\
\left. u \right|_{t=0} = u_0(x), \\
\left. u \right|_{x=0} = \mu_1(t), \\
\left. u \right|_{x=l_x} = \mu_2(t),
\end{array}$$
где $u(x, t)$ - функция температуры, $\alpha = const$ - коэффициент теплопроводности, $f(x, t)$ - функция источника. 

## Явная схема

Запишем разностное уравнение явной схемы:
$$\dfrac{y_i^{k+1} - y_i^{k}}{\tau} = \alpha \dfrac{y_{i+1}^{k} - 2 y_i^{k} + y_{i-1}^{k}}{h^2} + f_i^k,$$
где $\tau$ и $h$ - шаги по времени и пространству, $y_i^k$ - значение приближённого решения в точке $(i, k)$ сетки, $y_i^k$ - значение функции источника в той же точке сетки. 

Аппроксимируем начальное и граничные условия:
$$
\begin{array}{l}
y_i^0 = u_0(x_i), \; \forall i = \overline{0, N},\\
y_0^k = \mu_1(t_k), \\
y_N^k = \mu_2(t_k), \; \forall k > 0.
\end{array}$$

Запрограммируйте явную разностную схему решения начально-краевой задачи для однородного уравнения теплопроводности. Обратите внимание, что 
$$\exists \lim\limits_{t \rightarrow \infty} u(x, t) = u_\infty (x).$$
поэтому расчёт в какой-то момент следует остановить (считать до установления).

Во время проведения расчетов помните о том, что явная схема *условно* устойчива.

In [4]:
def heat_expl(init, bound1, bound2, alpha, lx, h, tau, tol=1e-3):
    """ Solve the heat equation `u_t = a*u_xx` for x in (0; lx) with an explicit scheme.
    
    Parameters
    ----------
    init : callable
       Initial condition
    bound1 : callable
       Boundary condition for x = 0
    bound1 : callable
       Boundary condition for x = lx
    alpha : float
       Thermal diffusivity
    h : float
       Spatial step
    tau : float
       Time step
    tol : float, optional
       Target tolerance.
       Stop iterations when the 2-norm of the difference between 
       solution on this time step and the next is less the tol.
       
    Returns
    -------
    t_end : float
       End time of calculation
    u_end : ndarray, shape (N,)
       Limit u_∞(x) (See above)
    """
    
    ### BEGIN SOLUTION
    N = int(lx / h)
    x = np.linspace(0, lx, N+1, endpoint=True)
    t_end = 0
    
    u_end = np.asarray([init(x_i) for x_i in x])

    u_prev = np.ones(N+1)
    u_prev[0] = bound1(t_end)
    u_prev[-1] = bound2(t_end)

    sigma = alpha * tau / h**2
    err = np.linalg.norm(u_prev - u_end)

    while (err > tol):
        t_end += tau
        err = np.linalg.norm(u_prev - u_end)
        u_prev = np.copy(u_end)

        for j in range(1, N):
            u_end[j] = u_prev[j] + sigma * (u_prev[j-1] + u_prev[j+1] - 2.*u_prev[j])

        u_end[0] = bound1(t_end)
        u_end[-1] = bound2(t_end)
        
    ### END SOLUTION
    
    return t_end, u_end

Протестируйте Вашу функцию.

In [5]:
from numpy.testing import assert_allclose

t_0, u_0 = heat_expl(lambda x: 0., lambda x: 1., lambda x: 1., 
                     alpha=1., lx=1., h=0.1, tau=0.005) 
assert_allclose(u_0, np.ones(11), atol=1e-2)

t_1, u_1 = heat_expl(lambda x: np.sin(4.*x), lambda x: 0., lambda x: 0., 
                     alpha=1., lx=np.pi, h=0.1, tau=0.005) 
assert_allclose(u_1, np.zeros(32), atol=1e-2)

### BEGIN HIDDEN TESTS
t_2, u_2 = heat_expl(lambda x: np.where(x <= 1, x + 3, x + 1), lambda x: 2., lambda x: 4., 
                     alpha=1., lx=2., h=0.1, tau=0.005, tol=1e-5) 

assert_allclose(u_2, 
                2 + np.linspace(0, 2, 21, endpoint=True), atol=1e-3)
### END HIDDEN TESTS

Определите порядки точности схемы (по пространству и времени) на тестовой задаче. Для этого для каждой переменной ($t$ или $x$):

1. Сделайте несколько расчётов для разных значений шага (например, $h_0, \; 2 h_0, \; 4 h_0$).
2. В один и тот же момент времени $t_1$ найдите ошибку полученных решений. Для этого либо возьмите аналитическое решение задачи, либо сравните результат в конечный момент времени, например, с решением в момент времени $0.99 t_1$. Обратите внимание, что имеющуюся функцию `heat_expl` надо немного модифицировать.
3. Найдите отношения этих ошибок. Сопоставьте полученные величины с порядком аппроксимации схемы по данной переменной.

In [4]:
# YOUR CODE HERE

## Неявная схема 

Запишем разностное уравнение неявной схемы:
$$\dfrac{y_i^{k+1} - y_i^{k}}{\tau} = \alpha \dfrac{y_{i+1}^{k+1} - 2 y_i^{k+1} + y_{i-1}^{k+1}}{h^2} + f_i^{k+1}.$$

Аппроксимировать начальное и граничные условия будем так же, как в случае явной схемы.

Запрограммируйте явную разностную схему решения начально-краевой задачи для однородного уравнения теплопроводности. Для решения системы линейных уравнений используйте встроенные функции `scipy`.

In [6]:
def heat_impl(init, bound1, bound2, alpha, lx, h, tau, tol=1e-3):
    """ Solve heat equation u_t = a*u_xx for x in (0; lx) with implicit scheme
    
    Parameters
    ----------
    init : callable
       Initial condition
    bound1 : callable
       Boundary condition for x = 0
    bound1 : callable
       Boundary condition for x = lx
    alpha : float, optional
       Thermal diffusivity
    h : float
       Spatial step
    tau : float
       Time step
    tol : float, optional
       Target tolerance.
       Stop iterations when the 2-norm of the difference between 
       solution on this time step and the next is less the tol.
       
    Returns
    -------
    t_end : float
       End time of calculation
    u_end : ndarray, shape (N,)
       Limit u_∞(x) (See above)
    """
    
    ### BEGIN SOLUTION
    from scipy.linalg import solve
    
    N = int(lx / h)
    x = np.linspace(0, lx, N+1, endpoint=True)
    t_end = 0
    
    u_end = np.asarray([init(x_i) for x_i in x])
    u_end[0] = bound1(t_end)
    u_end[-1] = bound2(t_end)

    u_prev = np.ones(N+1)
    u_prev[0] = bound1(t_end)
    u_prev[-1] = bound2(t_end)

    sigma = alpha * tau / h**2
    err = np.linalg.norm(u_prev - u_end)
    
    A = np.zeros([N+1,N+1])
    for i in range(1,N,1):
        A[i, i] = 1.0 + 2. * sigma
        A[i, i-1] = - sigma
        A[i, i+1] = - sigma
    A[0, 0]=1.
    A[N, N]=1.
    
    while (err > tol):
        t_end += tau
        err = np.linalg.norm(u_prev - u_end)
        u_prev = np.copy(u_end)
        u_end = solve(A, u_prev)
    
    ### END SOLUTION
        
    return t_end, u_end

Протестируйте Вашу функцию.

In [7]:
from numpy.testing import assert_allclose

t_0, u_0 = heat_impl(lambda x: 0., lambda x: 1., lambda x: 1., 
                     alpha=1., lx=1., h=0.1, tau=0.005) 
assert_allclose(u_0, np.ones(11), atol=1e-2)

t_1, u_1 = heat_impl(lambda x: np.sin(4.*x), lambda x: 0., lambda x: 0., 
                     alpha=1., lx=np.pi, h=0.1, tau=0.005) 
assert_allclose(u_1, np.zeros(32), atol=1e-2)

### BEGIN HIDDEN TESTS
t_2, u_2 = heat_impl(lambda x: np.where(x <= 1, x + 3, x + 1), lambda x: 2., lambda x: 4., 
                     alpha=1., lx=2., h=0.1, tau=0.005, tol=1e-5) 

assert_allclose(u_2, 
                2 + np.linspace(0, 2, 21, endpoint=True), atol=1e-3)
### END HIDDEN TESTS

Определите порядки точности схемы (по пространству и времени) на тестовой задаче. (см. выше)

In [ ]:
# YOUR CODE HERE